In [4]:
import numpy as np
from numpy.linalg import eig
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import scipy
import gzip
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score
from math import sqrt
import pickle
import gzip
import random


In [5]:
def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('mnist.pkl.gz', 'rb')
    u = pickle._Unpickler( f )
    u.encoding = 'latin1'
    training_data, validation_data, test_data = u.load()
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))
     
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
    
    
    def feedforward(self, a):
        """Return the output of the network if "a" is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The "training_data" is a list of tuples
        "(x, y)" representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If "test_data" is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print ("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))
                
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The "mini_batch" is a list of tuples "(x, y)", and "eta"
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
    
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y)*sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)
   

In [4]:
training_data, validation_data, test_data = load_data_wrapper()
np.shape(training_data)


(50000, 2)

In [6]:
net = Network([784, 30, 10])
net.weights

[array([[ 0.23155515,  0.95890853,  0.6603353 , ...,  0.04491267,
          0.19886506,  0.27484   ],
        [-1.13566212, -0.36167662,  0.46283329, ...,  0.49329918,
         -0.4204491 , -1.27314396],
        [-0.65757986, -0.96496181,  2.17387307, ..., -1.65442309,
         -1.32231804,  0.82016828],
        ...,
        [ 0.09485164, -0.04600972, -0.38701924, ..., -1.28352586,
         -0.62876946,  1.07868382],
        [ 0.74329737, -0.14976491,  0.02668591, ...,  2.44648968,
          0.90147871,  0.71622718],
        [-0.07155827,  0.40751246,  0.33212674, ..., -0.08507893,
         -1.49892449,  0.75309229]]),
 array([[ 1.15418613,  0.04350252, -1.46769707,  0.41992773, -1.01784448,
         -0.45198662, -0.04945125,  0.47191053,  1.62422822, -0.60295223,
          0.67428409,  0.54165012,  0.85151916,  0.96912156, -0.9757929 ,
          0.8343473 ,  1.15952319, -0.61966819, -1.10256639,  2.52859537,
          0.13630456, -0.27004115,  0.05568938, -1.06214086, -0.88211522,
   

In [60]:
net = Network([784, 30, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9068 / 10000
Epoch 1: 9243 / 10000
Epoch 2: 9222 / 10000
Epoch 3: 9287 / 10000
Epoch 4: 9324 / 10000
Epoch 5: 9416 / 10000
Epoch 6: 9403 / 10000
Epoch 7: 9386 / 10000
Epoch 8: 9456 / 10000
Epoch 9: 9458 / 10000
Epoch 10: 9466 / 10000
Epoch 11: 9476 / 10000
Epoch 12: 9454 / 10000
Epoch 13: 9490 / 10000
Epoch 14: 9502 / 10000
Epoch 15: 9474 / 10000
Epoch 16: 9487 / 10000
Epoch 17: 9493 / 10000
Epoch 18: 9511 / 10000
Epoch 19: 9495 / 10000
Epoch 20: 9511 / 10000
Epoch 21: 9518 / 10000
Epoch 22: 9459 / 10000
Epoch 23: 9510 / 10000
Epoch 24: 9481 / 10000
Epoch 25: 9506 / 10000
Epoch 26: 9492 / 10000
Epoch 27: 9513 / 10000
Epoch 28: 9533 / 10000
Epoch 29: 9533 / 10000


In [8]:
net = Network([784, 100, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9134 / 10000
Epoch 1: 9263 / 10000
Epoch 2: 9362 / 10000
Epoch 3: 9470 / 10000
Epoch 4: 9483 / 10000
Epoch 5: 9487 / 10000
Epoch 6: 9485 / 10000
Epoch 7: 9509 / 10000
Epoch 8: 9528 / 10000
Epoch 9: 9538 / 10000
Epoch 10: 9545 / 10000
Epoch 11: 9546 / 10000
Epoch 12: 9585 / 10000
Epoch 13: 9580 / 10000
Epoch 14: 9609 / 10000
Epoch 15: 9605 / 10000
Epoch 16: 9611 / 10000
Epoch 17: 9601 / 10000
Epoch 18: 9597 / 10000
Epoch 19: 9616 / 10000
Epoch 20: 9614 / 10000
Epoch 21: 9578 / 10000
Epoch 22: 9635 / 10000
Epoch 23: 9622 / 10000
Epoch 24: 9644 / 10000
Epoch 25: 9640 / 10000
Epoch 26: 9628 / 10000
Epoch 27: 9652 / 10000
Epoch 28: 9646 / 10000
Epoch 29: 9660 / 10000


In [62]:
net = Network([784, 100, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 4519 / 10000
Epoch 1: 4707 / 10000
Epoch 2: 5650 / 10000
Epoch 3: 6502 / 10000
Epoch 4: 6553 / 10000
Epoch 5: 6574 / 10000
Epoch 6: 6581 / 10000
Epoch 7: 6599 / 10000
Epoch 8: 6615 / 10000
Epoch 9: 6642 / 10000
Epoch 10: 6641 / 10000
Epoch 11: 6670 / 10000
Epoch 12: 6673 / 10000
Epoch 13: 6643 / 10000
Epoch 14: 6665 / 10000
Epoch 15: 6686 / 10000
Epoch 16: 6690 / 10000
Epoch 17: 6706 / 10000
Epoch 18: 6686 / 10000
Epoch 19: 6716 / 10000
Epoch 20: 6803 / 10000
Epoch 21: 7757 / 10000
Epoch 22: 7768 / 10000
Epoch 23: 7776 / 10000
Epoch 24: 7774 / 10000
Epoch 25: 7764 / 10000
Epoch 26: 7785 / 10000
Epoch 27: 7775 / 10000
Epoch 28: 7786 / 10000
Epoch 29: 7783 / 10000


In [63]:
net = Network([784, 50,30,30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9090 / 10000
Epoch 1: 9126 / 10000
Epoch 2: 9276 / 10000
Epoch 3: 9348 / 10000
Epoch 4: 9404 / 10000
Epoch 5: 9428 / 10000
Epoch 6: 9402 / 10000
Epoch 7: 9448 / 10000
Epoch 8: 9492 / 10000
Epoch 9: 9503 / 10000
Epoch 10: 9459 / 10000
Epoch 11: 9500 / 10000
Epoch 12: 9537 / 10000
Epoch 13: 9495 / 10000
Epoch 14: 9510 / 10000
Epoch 15: 9513 / 10000
Epoch 16: 9562 / 10000
Epoch 17: 9570 / 10000
Epoch 18: 9554 / 10000
Epoch 19: 9552 / 10000
Epoch 20: 9563 / 10000
Epoch 21: 9501 / 10000
Epoch 22: 9534 / 10000
Epoch 23: 9561 / 10000
Epoch 24: 9550 / 10000
Epoch 25: 9520 / 10000
Epoch 26: 9547 / 10000
Epoch 27: 9585 / 10000
Epoch 28: 9593 / 10000
Epoch 29: 9573 / 10000


In [5]:
net = Network([784, 50,30,30, 10])
net.SGD(training_data, 30, 10, 10, test_data=test_data)

Epoch 0: 7789 / 10000
Epoch 1: 8890 / 10000
Epoch 2: 8809 / 10000
Epoch 3: 9111 / 10000
Epoch 4: 9147 / 10000
Epoch 5: 9127 / 10000
Epoch 6: 9190 / 10000
Epoch 7: 9206 / 10000
Epoch 8: 9207 / 10000
Epoch 9: 9063 / 10000
Epoch 10: 9294 / 10000
Epoch 11: 9257 / 10000
Epoch 12: 9283 / 10000
Epoch 13: 9349 / 10000
Epoch 14: 9188 / 10000
Epoch 15: 9263 / 10000
Epoch 16: 9239 / 10000
Epoch 17: 9259 / 10000
Epoch 18: 9332 / 10000
Epoch 19: 9313 / 10000
Epoch 20: 9328 / 10000
Epoch 21: 9260 / 10000
Epoch 22: 9302 / 10000
Epoch 23: 9218 / 10000
Epoch 24: 9320 / 10000
Epoch 25: 9294 / 10000
Epoch 26: 9372 / 10000
Epoch 27: 9297 / 10000
Epoch 28: 9375 / 10000
Epoch 29: 9348 / 10000


In [6]:
net = Network([784,100, 50, 30, 10])
net.SGD(training_data, 30, 10, 10, test_data=test_data)

Epoch 0: 982 / 10000
Epoch 1: 1109 / 10000
Epoch 2: 1339 / 10000
Epoch 3: 2182 / 10000
Epoch 4: 2846 / 10000
Epoch 5: 3767 / 10000
Epoch 6: 4665 / 10000
Epoch 7: 7957 / 10000
Epoch 8: 8976 / 10000
Epoch 9: 8651 / 10000
Epoch 10: 8911 / 10000
Epoch 11: 9089 / 10000
Epoch 12: 9027 / 10000
Epoch 13: 9051 / 10000
Epoch 14: 9062 / 10000
Epoch 15: 9086 / 10000
Epoch 16: 8974 / 10000
Epoch 17: 9000 / 10000
Epoch 18: 9213 / 10000
Epoch 19: 9204 / 10000
Epoch 20: 9180 / 10000
Epoch 21: 9210 / 10000
Epoch 22: 9232 / 10000
Epoch 23: 8989 / 10000
Epoch 24: 8834 / 10000
Epoch 25: 9253 / 10000
Epoch 26: 9073 / 10000
Epoch 27: 9167 / 10000
Epoch 28: 9122 / 10000
Epoch 29: 9141 / 10000
